### Importamos librerías


In [ ]:
try:
    import pandas as pd
    print("Libreria: 'pandas' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'pandas' no esta instalada, se debe instalar")
    !pip install pandas
    import pandas as pd
try:
    import numpy as np
    print("Libreria: 'numpy' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'numpy' no esta instalada, se debe instalar")
    !pip install numpy
    import numpy as np
try:
    from ydata_profiling import ProfileReport
    print("Libreria: 'ProfileReport' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'ProfileReport' no esta instalada, se debe instalar")
    !pip install ProfileReport
    from ydata_profiling import ProfileReport

### Funciones


In [ ]:
def clonar_repo():
  !git clone https://github.com/leandroomargarcia/Gero_Lean_Fran_Ale_Rafa_AutoML.git

def actualizar_repo():
    # Cambia al directorio del repositorio clonado
    %cd Gero_Lean_Fran_Ale_Rafa_AutoML

    # Realiza un pull para obtener los cambios más recientes del repositorio
    !git pull origin main  # Cambia "main" si estás en otra rama

def cargar_dataset():
  ruta_csv = '/content/Gero_Lean_Fran_Ale_Rafa_AutoML/Datasetv4.csv'

  # Cargar el archivo CSV en un DataFrame de pandas
  dataframe = pd.read_csv(ruta_csv ,sep = ';', on_bad_lines='skip')
  return dataframe

def cargar_paises():
  ruta_csv = '/content/Gero_Lean_Fran_Ale_Rafa_AutoML/JOIN_PAISES.csv'

  # Cargar el archivo CSV en un DataFrame de pandas
  dataframe = pd.read_csv(ruta_csv ,sep = ';', on_bad_lines='skip')
  return dataframe

def evaluar_nan(dataframe):
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd.set_option('display.max_info_rows', None)
  resultado = dataframe.isna().sum()
  print(resultado)

def generar_tabla_auxiliar_target(df):
  df_aux = df.iloc[:, [0,1,27]]
  # Convert CALENDAR_DATE to datetime format
  df_aux['CALENDAR_DATE'] = pd.to_datetime(df_aux['CALENDAR_DATE'], format='%d/%m/%Y')

  # Add one month and create a new column
  df_aux['CALENDAR_DATE_LAG_1'] = df_aux['CALENDAR_DATE'] + pd.DateOffset(months=-1)
  df_aux['CALENDAR_DATE_LAG_1'] = df_aux['CALENDAR_DATE_LAG_1'].dropna()
  # Format the date and convert to int
  df_aux['CALENDAR_DATE_LAG_1'] = df_aux['CALENDAR_DATE_LAG_1'].dt.strftime('%Y%m').astype(int)

  df_aux.rename(columns={'CANT_CASOS': 'Target'}, inplace=True)
  df_aux = df_aux.drop('CALENDAR_DATE', axis = 1) # Eliminamos la fecha original
  return df_aux

def eliminar_fechas_faltantes(df):
  print(f"Nulos CALENDAR antes: {df['CALENDAR_DATE'].isna().sum()}")
  df = df[~df["CALENDAR_DATE"].isna()]
  print(f"Nulos CALENDAR después: {df['CALENDAR_DATE'].isna().sum()}")
  return df

def generar_tabla_predictivas(df):
  df["CALENDAR_DATE"] = pd.to_datetime(df['CALENDAR_DATE'], format='%d/%m/%Y')
  # Format the date and convert to int
  df['CALENDAR_DATE'] = df['CALENDAR_DATE'].dt.strftime('%Y%m').astype(int)
  return df

def realizar_join_predictivas_y_target(df_aux, predictivas):
  # Merge the DataFrames based on the specified keys
  merged_df = predictivas.merge(df_aux, how='left', left_on=['CUS_CUST_ID_SEL', 'CALENDAR_DATE'], right_on=['CUS_CUST_ID_SEL', 'CALENDAR_DATE_LAG_1'])
  merged_df = merged_df.drop(['CALENDAR_DATE_LAG_1','CANT_CASOS'], axis = 1)
  merged_df = merged_df[merged_df["CALENDAR_DATE"]  != pd.to_datetime("31/08/2023", format='%d/%m/%Y')]
  merged_df = merged_df[~merged_df["Target"].isna()]
  return merged_df

### Análisis Exploratorio

In [ ]:
# Llama a la función para clonar el repositorio
clonar_repo()

fatal: destination path 'Gero_Lean_Fran_Ale_Rafa_AutoML' already exists and is not an empty directory.


In [ ]:
# Llama a la función para actualizar el repositorio
actualizar_repo()

/content/Gero_Lean_Fran_Ale_Rafa_AutoML
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 77.95 KiB | 7.09 MiB/s, done.
From https://github.com/leandroomargarcia/Gero_Lean_Fran_Ale_Rafa_AutoML
 * branch            main       -> FETCH_HEAD
   68ddcd7..f1ef4a8  main       -> origin/main
Updating 68ddcd7..f1ef4a8
Fast-forward
 JOIN PAISES.csv | 13946 ++++++++++++++++++++++++++++++++++++++++++++++++++++++
 JOIN_PAISES.csv | 13946 ++++++++++++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 27892 insertions(+)
 create mode 100644 JOIN PAISES.csv
 create mode 100644 JOIN_PAISES.csv


In [ ]:
df = cargar_dataset()
paises = cargar_paises()

In [ ]:
df["CANT_CASOS"].replace(np.NaN, 0, inplace=True)
df["CANT_CASOS"] = df["CANT_CASOS"].apply(lambda x: 1 if x > 0 else 0)
df["CANT_CASOS"].value_counts()

0    141781
1      7317
Name: CANT_CASOS, dtype: int64

In [ ]:
evaluar_nan(df)

CALENDAR_DATE                            768
CUS_CUST_ID_SEL                            0
SIT_SITE_ID                            56881
CUS_FIRST_SALE_DATE                       54
SELLER_SEGMENT                             0
SELLER_SEGMENT_DETAIL                      0
TOP_VERTICAL                           10442
TOP_AGG1                               10442
TOP_AGG2                               10444
TOP_DOM_MODA                          108070
SKU_KEY_NOT_SALEABLE_STOCK               768
SKU_INP_TOTAL_STOCK                      768
SKU_QUARENTINE_STOCK                     768
SKU_TRANSFER_STOCK                       768
SKU_TOTAL_STOCK                          768
SKU_TOTAL_THEOR_STOCK                    768
SKU_AGING_UPPER_60D_STOCK                768
SKU_AGING_UPPER_90D_STOCK                768
SKU_AGING_UPPER_120D_STOCK               768
SKU_AGING_UPPER_180D_STOCK               768
SKU_SI_FBM_L30                           768
SKU_SI_FBM_L60                           768
SKU_SI_FBM

In [ ]:
df = eliminar_fechas_faltantes(df)

Nulos CALENDAR antes: 768
Nulos CALENDAR después: 0


In [ ]:
df["CALENDAR_DATE"].value_counts()

31/1/2023     13935
31/12/2022    13869
30/11/2022    13713
28/2/2023     13542
31/3/2023     13431
30/4/2023     13364
31/10/2022    13346
31/5/2023     13330
30/6/2023     13312
31/7/2023     13275
31/8/2023     13213
Name: CALENDAR_DATE, dtype: int64

In [ ]:
filtered_df_11 = df.groupby('CUS_CUST_ID_SEL').filter(lambda x: len(x) == 11)
filtered_df_10 = df.groupby('CUS_CUST_ID_SEL').filter(lambda x: len(x) == 11)
filtered_df_9 = df.groupby('CUS_CUST_ID_SEL').filter(lambda x: len(x) == 11)

print(filtered_df_11.shape)
print(filtered_df_10.shape)
print(filtered_df_9.shape)

(137940, 37)
(137940, 37)
(137940, 37)


In [ ]:
df_aux = generar_tabla_auxiliar_target(df)

<ipython-input-114-cee8cb78788c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['CALENDAR_DATE'] = pd.to_datetime(df_aux['CALENDAR_DATE'], format='%d/%m/%Y')
<ipython-input-114-cee8cb78788c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aux['CALENDAR_DATE_LAG_1'] = df_aux['CALENDAR_DATE'] + pd.DateOffset(months=-1)
<ipython-input-114-cee8cb78788c>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [ ]:
df_aux.head(5)

,CUS_CUST_ID_SEL,Target,CALENDAR_DATE_LAG_1
0,1000325486,0,202301
1,10003525,0,202301
2,1000658032,0,202301
3,1000980425,0,202301
4,1000999989,0,202301


In [ ]:
predictivas = generar_tabla_predictivas(df)
predictivas.head(10)

,CALENDAR_DATE,CUS_CUST_ID_SEL,SIT_SITE_ID,CUS_FIRST_SALE_DATE,SELLER_SEGMENT,SELLER_SEGMENT_DETAIL,TOP_VERTICAL,TOP_AGG1,TOP_AGG2,TOP_DOM_MODA,SKU_KEY_NOT_SALEABLE_STOCK,SKU_INP_TOTAL_STOCK,SKU_QUARENTINE_STOCK,SKU_TRANSFER_STOCK,SKU_TOTAL_STOCK,SKU_TOTAL_THEOR_STOCK,SKU_AGING_UPPER_60D_STOCK,SKU_AGING_UPPER_90D_STOCK,SKU_AGING_UPPER_120D_STOCK,SKU_AGING_UPPER_180D_STOCK,SKU_SI_FBM_L30,SKU_SI_FBM_L60,SKU_SI_FBM_L90,SKU_KEY_NOT_SELLING_ME2_30D_STOCK,SKU_KEY_NOT_SELLING_ME2_60D_STOCK,SKU_KEY_NOT_SELLING_ME2_90D_STOCK,SKU_KEY_NOT_SELLING_ME2_120D_STOCK,CANT_CASOS,CANT_REQUEST,CANT_BOXES,CANT_BOXES_ENE,CANT_BOXES_DELAY,CANT_BOXES_STALE,UNIDADES_RESERVADAS,UNIDADES_PREPARADAS,VALOR_USD_RESERVADO,VOLUMEN_RESERVADO
0,202302,1000325486,NaN,44487.0,LONG TAIL,NO ASESORADO,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,16.0,17.0,15.0,15.0,15.0,15.0,5.0,6.0,12.0,7.0,7.0,7.0,2.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,202302,10003525,MLB,44454.0,TO,NO ASESORADO,ACC,AUTOPARTS,REPLACEMENT AUTOPARTS,Accesorios,0.0,0.0,0.0,18.0,1299.0,1270.0,294.0,60.0,9.0,3.0,2798.0,5354.0,8382.0,78.0,11.0,5.0,2.0,0,1,1,0,0,0,1.0,1.0,2.767328e+09,2998475.0
2,202302,1000658032,NaN,44488.0,MIDTAIL,MT ASESORADO,T & B,TOYS AND GAMES,DOLLS & ACTION FIGURES,NaN,3.0,0.0,0.0,0.0,360.0,362.0,6.0,0.0,0.0,0.0,227.0,482.0,782.0,22.0,20.0,20.0,20.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,202302,1000980425,NaN,44491.0,LONG TAIL,NO ASESORADO,APP & SPORTS,SPORTS SUPPLEMENTS,SPORTS SUPPLEMENTS,NaN,0.0,0.0,0.0,0.0,125.0,136.0,123.0,122.0,118.0,0.0,12.0,13.0,18.0,106.0,106.0,96.0,35.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,202302,1000999989,NaN,44673.0,LONG TAIL,NO ASESORADO,APP & SPORTS,APPAREL,BASICS,Interior,0.0,0.0,0.0,0.0,506.0,512.0,74.0,35.0,25.0,0.0,279.0,652.0,1068.0,28.0,6.0,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
5,202302,1001014434,MLB,44488.0,LONG TAIL,NO ASESORADO,APP & SPORTS,APPAREL,CLOTHING,Tops,0.0,0.0,0.0,0.0,451.0,424.0,137.0,124.0,107.0,42.0,439.0,937.0,1639.0,121.0,95.0,56.0,51.0,0,1,1,0,0,0,1.0,1.0,1.210232e+07,2550562.0
6,202302,1001027358,NaN,44484.0,LONG TAIL,NO ASESORADO,APP & SPORTS,APPAREL ACCESSORIES,OTHER APPAREL ACCESSORIES,NaN,0.0,0.0,0.0,0.0,1623.0,1623.0,169.0,65.0,25.0,16.0,860.0,1426.0,1963.0,305.0,205.0,184.0,176.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
7,202302,1001253,NaN,44454.0,MIDTAIL,MT ASESORADO,HOME & INDUSTRY,LIGHTING,LAMPS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
8,202302,1001257705,MLB,44543.0,LONG TAIL,NO ASESORADO,HOME & INDUSTRY,HOUSEHOLD ITEMS,TABLETOP,NaN,10.0,0.0,0.0,0.0,232.0,232.0,0.0,0.0,0.0,0.0,215.0,341.0,362.0,32.0,32.0,32.0,22.0,0,1,0,0,0,0,9.0,0.0,3.444237e+09,145233.0
9,202302,1002079450,NaN,44497.0,MIDTAIL,MT ASESORADO,CE,FURNITURE & MATTRESSES,OFFICE FURNITURE,NaN,0.0,0.0,0.0,0.0,48.0,49.0,4.0,2.0,0.0,0.0,485.0,736.0,979.0,14.0,5.0,5.0,5.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [ ]:
# Realizamos el join entre las variables predictivas y la target con LAG de un mes
df_final = realizar_join_predictivas_y_target(df_aux, predictivas)

In [ ]:
df_final.head(5)

,CALENDAR_DATE,CUS_CUST_ID_SEL,SIT_SITE_ID,CUS_FIRST_SALE_DATE,SELLER_SEGMENT,SELLER_SEGMENT_DETAIL,TOP_VERTICAL,TOP_AGG1,TOP_AGG2,TOP_DOM_MODA,SKU_KEY_NOT_SALEABLE_STOCK,SKU_INP_TOTAL_STOCK,SKU_QUARENTINE_STOCK,SKU_TRANSFER_STOCK,SKU_TOTAL_STOCK,SKU_TOTAL_THEOR_STOCK,SKU_AGING_UPPER_60D_STOCK,SKU_AGING_UPPER_90D_STOCK,SKU_AGING_UPPER_120D_STOCK,SKU_AGING_UPPER_180D_STOCK,SKU_SI_FBM_L30,SKU_SI_FBM_L60,SKU_SI_FBM_L90,SKU_KEY_NOT_SELLING_ME2_30D_STOCK,SKU_KEY_NOT_SELLING_ME2_60D_STOCK,SKU_KEY_NOT_SELLING_ME2_90D_STOCK,SKU_KEY_NOT_SELLING_ME2_120D_STOCK,CANT_REQUEST,CANT_BOXES,CANT_BOXES_ENE,CANT_BOXES_DELAY,CANT_BOXES_STALE,UNIDADES_RESERVADAS,UNIDADES_PREPARADAS,VALOR_USD_RESERVADO,VOLUMEN_RESERVADO,Target
0,202302,1000325486,NaN,44487.0,LONG TAIL,NO ASESORADO,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,16.0,17.0,15.0,15.0,15.0,15.0,5.0,6.0,12.0,7.0,7.0,7.0,2.0,0,0,0,0,0,NaN,NaN,NaN,NaN,0.0
1,202302,10003525,MLB,44454.0,TO,NO ASESORADO,ACC,AUTOPARTS,REPLACEMENT AUTOPARTS,Accesorios,0.0,0.0,0.0,18.0,1299.0,1270.0,294.0,60.0,9.0,3.0,2798.0,5354.0,8382.0,78.0,11.0,5.0,2.0,1,1,0,0,0,1.0,1.0,2.767328e+09,2998475.0,0.0
2,202302,1000658032,NaN,44488.0,MIDTAIL,MT ASESORADO,T & B,TOYS AND GAMES,DOLLS & ACTION FIGURES,NaN,3.0,0.0,0.0,0.0,360.0,362.0,6.0,0.0,0.0,0.0,227.0,482.0,782.0,22.0,20.0,20.0,20.0,0,0,0,0,0,NaN,NaN,NaN,NaN,0.0
3,202302,1000980425,NaN,44491.0,LONG TAIL,NO ASESORADO,APP & SPORTS,SPORTS SUPPLEMENTS,SPORTS SUPPLEMENTS,NaN,0.0,0.0,0.0,0.0,125.0,136.0,123.0,122.0,118.0,0.0,12.0,13.0,18.0,106.0,106.0,96.0,35.0,0,0,0,0,0,NaN,NaN,NaN,NaN,0.0
4,202302,1000999989,NaN,44673.0,LONG TAIL,NO ASESORADO,APP & SPORTS,APPAREL,BASICS,Interior,0.0,0.0,0.0,0.0,506.0,512.0,74.0,35.0,25.0,0.0,279.0,652.0,1068.0,28.0,6.0,0.0,0.0,0,0,0,0,0,NaN,NaN,NaN,NaN,0.0


In [ ]:
df_final["CALENDAR_DATE"].value_counts()

202212    13867
202211    13709
202301    13541
202302    13411
202210    13338
202303    13336
202304    13281
202305    13265
202306    13258
202307    13185
Name: CALENDAR_DATE, dtype: int64

In [ ]:
evaluar_nan(df_final)

CALENDAR_DATE                             0
CUS_CUST_ID_SEL                           0
SIT_SITE_ID                           49849
CUS_FIRST_SALE_DATE                      40
SELLER_SEGMENT                            0
SELLER_SEGMENT_DETAIL                     0
TOP_VERTICAL                           9187
TOP_AGG1                               9187
TOP_AGG2                               9189
TOP_DOM_MODA                          97219
SKU_KEY_NOT_SALEABLE_STOCK                0
SKU_INP_TOTAL_STOCK                       0
SKU_QUARENTINE_STOCK                      0
SKU_TRANSFER_STOCK                        0
SKU_TOTAL_STOCK                           0
SKU_TOTAL_THEOR_STOCK                     0
SKU_AGING_UPPER_60D_STOCK                 0
SKU_AGING_UPPER_90D_STOCK                 0
SKU_AGING_UPPER_120D_STOCK                0
SKU_AGING_UPPER_180D_STOCK                0
SKU_SI_FBM_L30                            0
SKU_SI_FBM_L60                            0
SKU_SI_FBM_L90                  

In [ ]:
df_final.to_csv("./Dataset_v5_country_target_and_date_lag.csv", index=False)

In [ ]:
from ydata_profiling import ProfileReport
report = ProfileReport(df_final)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]